<a href="https://colab.research.google.com/github/tasheera/CVD_Risk_Analyser/blob/MachineLearning/CVD_predictions_logisticRegression.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [338]:
import pandas as pd
import numpy as np
import matplotlib
from matplotlib import pyplot as plt
%matplotlib inline
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split

#import dataset
cvd_data = pd.read_csv('health_data.csv', index_col = 'id')

cvd_data.head()

,age,gender,height,weight,bmi,bmi_status,systolic_BP,diastolic_BP,cholesterol,glucose,smoking,alcohol,active,cardio
id,,,,,,,,,,,,,,
0,50,1,168,62.0,21.97,2,110,80,0,0,0,0,1,0
1,55,0,156,85.0,34.93,4,140,90,2,0,0,0,1,1
2,52,0,165,64.0,23.51,2,130,70,2,0,0,0,0,1
3,48,1,169,82.0,28.71,3,150,100,0,0,0,0,1,1
4,48,0,156,56.0,23.01,2,100,60,0,0,0,0,0,0


In [339]:
cvd_data = cvd_data[["age",	"gender",	"height",	"weight",	"bmi",	"bmi_status",	"systolic_BP",	"diastolic_BP",	"cholesterol",	"glucose",	"smoking",	"alcohol",	"active", "cardio"]]


In [340]:
cvd_data.corr()["cardio"]

age             0.237749
gender          0.008109
height         -0.010821
weight          0.181660
bmi             0.165633
bmi_status      0.184964
systolic_BP     0.054475
diastolic_BP    0.065719
cholesterol     0.221147
glucose         0.089307
smoking        -0.015486
alcohol        -0.007330
active         -0.035653
cardio          1.000000
Name: cardio, dtype: float64

In [341]:
x = cvd_data.drop("cardio", axis=1)
y = cvd_data['cardio']

In [342]:
"""
# Define a function to remove outliers using IQR method
def remove_outliers_iqr(df, column):
    Q1 = df[column].quantile(0.25)
    Q3 = df[column].quantile(0.75)
    IQR = Q3 - Q1
    lower_bound = Q1 - 1.5 * IQR
    upper_bound = Q3 + 1.5 * IQR
    df[column] = df[column].clip(lower=lower_bound, upper=upper_bound)

# Apply outlier removal using IQR method for each feature
columns_to_process = ["height", "weight","bmi", "systolic_BP", "diastolic_BP"]
for column in columns_to_process:
    remove_outliers_iqr(cvd_data, column)
"""

'\n# Define a function to remove outliers using IQR method\ndef remove_outliers_iqr(df, column):\n    Q1 = df[column].quantile(0.25)\n    Q3 = df[column].quantile(0.75)\n    IQR = Q3 - Q1\n    lower_bound = Q1 - 1.5 * IQR\n    upper_bound = Q3 + 1.5 * IQR\n    df[column] = df[column].clip(lower=lower_bound, upper=upper_bound)\n\n# Apply outlier removal using IQR method for each feature\ncolumns_to_process = ["height", "weight","bmi", "systolic_BP", "diastolic_BP"]\nfor column in columns_to_process:\n    remove_outliers_iqr(cvd_data, column)\n'

In [343]:
# Calculate mean and standard deviation for the feature of interest (e.g., 'numerical_column1')
mean_h = cvd_data["height"].mean()
sd_h = cvd_data["height"].std()

# Define upper and lower limits
upper_limit_h = mean_h + (3 * sd_h)
lower_limit_h = mean_h - (3 * sd_h)

# Replace outliers with either min or max standard deviation value
cvd_data.loc[cvd_data["height"] > upper_limit_h, "height"] = upper_limit_h
cvd_data.loc[cvd_data["height"] < lower_limit_h, "height"] = lower_limit_h


# Calculate mean and standard deviation for the feature of interest (e.g., 'numerical_column1')
mean_w = cvd_data["weight"].mean()
sd_w = cvd_data["weight"].std()

# Define upper and lower limits
upper_limit_w = mean_w + (3 * sd_w)
lower_limit_w = mean_w - (3 * sd_w)

# Replace outliers with either min or max standard deviation value
cvd_data.loc[cvd_data["weight"] > upper_limit_w, "weight"] = upper_limit_w
cvd_data.loc[cvd_data["weight"] < lower_limit_w, "weight"] = lower_limit_w


# Calculate mean and standard deviation for the feature of interest (e.g., 'numerical_column1')
mean_sbp = cvd_data["systolic_BP"].mean()
sd_sbp = cvd_data["systolic_BP"].std()

# Define upper and lower limits
upper_limit_sbp = mean_sbp + (3 * sd_sbp)
lower_limit_sbp = mean_sbp - (3 * sd_sbp)

# Replace outliers with either min or max standard deviation value
cvd_data.loc[cvd_data["systolic_BP"] > upper_limit_sbp, "systolic_BP"] = upper_limit_sbp
cvd_data.loc[cvd_data["systolic_BP"] < lower_limit_sbp, "systolic_BP"] = lower_limit_sbp


# Calculate mean and standard deviation for the feature of interest (e.g., 'numerical_column1')
mean_dbp = cvd_data["diastolic_BP"].mean()
sd_dbp = cvd_data["diastolic_BP"].std()

# Define upper and lower limits
upper_limit_dbp = mean_dbp + (3 * sd_dbp)
lower_limit_dbp = mean_dbp - (3 * sd_dbp)

# Replace outliers with either min or max standard deviation value
cvd_data.loc[cvd_data["diastolic_BP"] > upper_limit_dbp, "diastolic_BP"] = upper_limit_dbp
cvd_data.loc[cvd_data["diastolic_BP"] < lower_limit_dbp, "diastolic_BP"] = lower_limit_dbp

In [344]:
from sklearn.model_selection import train_test_split

x_train, x_test, y_train, y_test = train_test_split(x,y,test_size=0.2,random_state=100)

In [345]:
"""
# Removing outliers
z_scores = (x_train[numerical_columns] - x_train[numerical_columns].mean()) / x_train[numerical_columns].std()
threshold = 3
outlier_indices = np.where((np.abs(z_scores) > threshold).any(axis=1))[0]

# Drop only valid indices
valid_indices = np.intersect1d(outlier_indices, x_train.index)
x_train_no_outliers = x_train.drop(index=valid_indices)
y_train_no_outliers = y_train.drop(index=valid_indices)
"""

'\n# Removing outliers\nz_scores = (x_train[numerical_columns] - x_train[numerical_columns].mean()) / x_train[numerical_columns].std()\nthreshold = 3\noutlier_indices = np.where((np.abs(z_scores) > threshold).any(axis=1))[0]\n\n# Drop only valid indices\nvalid_indices = np.intersect1d(outlier_indices, x_train.index)\nx_train_no_outliers = x_train.drop(index=valid_indices)\ny_train_no_outliers = y_train.drop(index=valid_indices)\n'

In [346]:
from sklearn.linear_model import LogisticRegression
log_reg = LogisticRegression()
log_reg.fit(x_train, y_train)

/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


LogisticRegression()

In [347]:
y_log_reg_train_prediction = log_reg.predict(x_train)
y_log_reg_test_prediction = log_reg.predict(x_test)
y_log_reg_test_prediction

array([1, 0, 0, ..., 1, 0, 1])

In [348]:
from sklearn.metrics import mean_squared_error, r2_score

log_reg_train_mse = mean_squared_error(y_train,y_log_reg_train_prediction)
log_reg_train_r2 = r2_score(y_train,y_log_reg_train_prediction)

log_reg_test_mse = mean_squared_error(y_test,y_log_reg_test_prediction)
log_reg_test_r2 = r2_score(y_test,y_log_reg_test_prediction)


print('log_reg MSE TRAIN : ',log_reg_train_mse)
print('log_reg R2 TRAIN : ',log_reg_train_r2)
print('log_reg MSE Test : ',log_reg_test_mse)
print('log_reg R2 Test : ',log_reg_test_r2)

log_reg MSE TRAIN :  0.27675
log_reg R2 TRAIN :  -0.10700381803357639
log_reg MSE Test :  0.2797142857142857
log_reg R2 Test :  -0.11887908658780044


In [349]:
from sklearn import metrics
#42 - 69.8
#100 - 70.5
#0 - 69.6
#2022 - 69.4
print("Logistic regression model accuracy(in %):", metrics.accuracy_score(y_test, y_log_reg_test_prediction)*100)


Logistic regression model accuracy(in %): 72.02857142857142


In [350]:
import joblib
joblib.dump(log_reg, 'logistic_regression_model.pkl')

['logistic_regression_model.pkl']

In [351]:
#numerical_columns = ["age",	"height",	"weight",	"bmi",	"systolic_BP",	"diastolic_BP"]
#scaler = StandardScaler()
#cvd_data[numerical_columns] = scaler.fit_transform(cvd_data[numerical_columns])

In [352]:
#categorical_columns = ["gender", "bmi_status", "cholesterol",	"glucose"]
#cvd_data = pd.get_dummies(cvd_data, columns=categorical_columns)